In [1]:
from premise import *
from datapackage import Package
import bw2data, bw2io, bw2calc
from unfold import Unfold
bw2data.projects.set_current("new4")

In [2]:
clear_cache()

Cache folder cleared!


In [3]:
combinations = [
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 5, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":2, "duration":False, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":True, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 1, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 2, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 3, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 0, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":True, "lead time":False, "capital replacement rate":False, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00},
    {"range time":False, "duration":20, "foresight":False, "lead time":False, "capital replacement rate":True, "measurement": 4, "weighted slope start": 0.75, "weighted slope end": 1.00}
]

In [ ]:
scenarios = [
    {"model": "remind", "pathway":"SSP2-Base", "year": 2050},
    {"model": "image", "pathway":"SSP2-RCP19", "year": 2048},
]

for model in [
    "cutoff",
    "consequential"
]:

    for ei in [
        #"3.6",
        #"3.7",
        #"3.7.1",
        #"3.8",
        #"3.9",
        "3.9.1"

    ]:
        
        if ei not in ["3.9", "3.9.1"]:
            bw2data.projects.set_current("new4")
        else:
            bw2data.projects.set_current("ei39")

        for source in [
            "brightway",
            #"ecospold"
        ]:
            for export in [
                "brightway",
                "superstructure",
                "simapro",
                #"matrices",
                #"datapackage"
            ]:

                ndb = NewDatabase(
                        scenarios = scenarios,
                        source_db= f"ecoinvent {ei} {model}" if source == "brightway" else None,
                        source_type=source,
                        source_file_path=f"/Users/romain/Documents/ecoinvent {ei}_cutoff_ecoSpold02/datasets" if source == "ecospold" else None, # <-- this is NEW
                        source_version=ei,
                        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
                        system_model=model,
                        system_args=combinations[0],
                        additional_inventories= [ # <-- this is NEW
                            {"filepath": "/Users/romain/GitHub/premise/premise/data/additional_inventories/lci-PV.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
                    ] # <-- this is NEW
                )
                

                print("#############################################################################################################")
                print(ei, source, export)

                ndb.update()
                ndb.update([
                    "buses",
                    "cars",
                ])

                if export == "brightway":
                        
                    ndb.write_db_to_brightway([
                        "test1",
                        "test2"
                        ]
                    )
                    lca = bw2calc.LCA({bw2data.Database("test1").random(): 1})
                    lca.lci()
                    lca = bw2calc.LCA({bw2data.Database("test2").random(): 1})
                    lca.lci()
                elif export == "superstructure":
                    ndb.write_superstructure_db_to_brightway("test_super")
                elif export == "simapro":
                    ndb.write_db_to_simapro()
                elif export == "matrices":
                    ndb.write_db_to_matrices()
                else:
                    ndb.write_datapackage(name="test")
                    u = Unfold("export/datapackage/test.zip")
                    u.unfold(dependencies={"ecoinvent": f"ecoinvent {ei} {model}", "biosphere3": "biosphere3"}, superstructure=True)

premise v.(2, 0, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

100%|█████████████████████████████████| 21238/21238 [00:00<00:00, 160393.08it/s]

100%|████████████████████████████████| 674593/674593 [01:01<00:00, 10951.89it/s]

100%|███████████████████████████████████| 21238/21238 [00:03<00:00, 6844.79it/s]
Extracting IAM data: : 4it [01:54, 28.64s/it]                         


#############################################################################################################
3.9.1 brightway brightway
`update()` will skip the following sectors: 'buses', 'cars', 'two_wheelers'.
If you want to update these sectors, please run them separately afterwards.


Updating: external:  90%|████████████▌ | 9/10 [07:25<00:49, 49.50s/it]


External scenarios (datapackages) are not provided. Skipped.
Done!



...

No markets found for bus in IAM data. Skipping.


Updating: cars: 100%|███████████████████| 2/2 [00:44<00:00, 22.41s/it]

No markets found for car in IAM data. Skipping.
Done!



Write new database(s) to Brightway.
Running all checks...
Anomalies found: check the change report.
Running all checks...
Database test1 already exists: it will be overwritten.
Vacuuming database 


Writing activities to SQLite3 database:
0% [#####################         ] 100% | ETA: 00:00:14

In [ ]:
p 